In [ ]:
import pandas as pd
import numpy as np
from datetime import date

pd.set_option('display.max_columns', 32)
pd.options.display.float_format = '{:.5f}'.format

In [ ]:
df = pd.read_csv('./flights.csv', low_memory=False)

In [ ]:
def null_info(dataframe):
    tab_info = pd.DataFrame(dataframe.dtypes).T.rename(index={0:'column type'})
    tab_info = tab_info.append(pd.DataFrame(dataframe.isnull().sum()).T.rename(index={0:'null values (nb)'}))
    tab_info = tab_info.append(pd.DataFrame(dataframe.isnull().sum()/dataframe.shape[0]*100).T.rename(index={0:'null values (%)'}))

    return tab_info

# II. Analysis
The dataset used in this project comes from the U.S. Department of Transportation, it is available from Kaggle and contains the records for a year of flights. The dataframe contains 581+ millions of datapoints with 31 features each, which will be studied and used based on the information gain each one provides.

### II.1 General information about the data

In [ ]:
print "The dataframe contains {} data points with {} features each.".format(*df.shape)

##### Brief description of the features
**YEAR**: Year of the Flight Trip  
**MONTH**: Month of the Flight Trip  
**DAY**: Day of the Flight Trip  
**DAY_OF_WEEK**: Day of week of the Flight Trip  
**AIRLINE**: Airline Identifier  
**FLIGHT_NUMBER**: Flight Identifier  
**TAIL_NUMBER**: Aircraft Identifier  
**ORIGIN_AIRPORT**: Starting Airport  
**DESTINATION_AIRPORT**: Destination Airport  
**SCHEDULED_DEPARTURE**: Planned Departure Time  
**DEPARTURE_TIME**: WHEEL_OFF - TAXI_OUT  
**DEPARTURE_DELAY**: Total Delay on Depature  
**TAXI_OUT**: The time duration elapsed between departure from the origin airport gate and wheels off  
**WHEELS_OFF**: The time point that the aircraft's wheels leave the ground  
**SCHEDULED_TIME**: Planned time amount needed for the flight trip  
**ELAPSED_TIME**: AIR_TIME + TAXI_IN + TAXI_OUT  
**AIR_TIME**: The time duration between wheels_off and wheels_on time  
**DISTANCE**: Distance between two airports  
**WHEELS_ON**: The time point that the aircraft's wheels touch on the ground  
**TAXI_IN**: The time duration elapsed between wheels-on and gate arrival at the destination airport  
**SCHEDULED_ARRIVAL**: Planned arrival time  
**ARRIVAL_TIME**: WHEELS_ON + TAXI_IN  
**ARRIVAL_DELAY**: ARRIVAL_TIME - SCHEDULED_ARRIVAL  
**DIVERTED**: Aircraft landed on airport that out of schedule  
**CANCELLED**: Flight Cancelled (1 = cancelled)  
**CANCELLATION_REASON**: Reason for Cancellation of flight: A - Airline/Carrier; B - Weather; C - National Air System; D - Security  
**AIR_SYSTEM_DELAY**: Delay caused by air system  
**SECURITY_DELAY**: Delay caused by security  
**AIRLINE_DELAY**: Delay caused by the airline  
**LATE_AIRCRAFT_DELAY**: Delay caused by aircraft  
**WEATHER_DELAY**: Delay caused by weather  

In [ ]:
null_info(df)

##### Columns that contain null values
Tail Number = 0.252978%  
Departure Time, Departure Delay = 1.48053%  
Taxi Out, Wheels Off = 1.53026%  
Scheduled Time = 0.000103%  
Wheels On, Taxi In, Arrival Time = 1.58982%  
Air Time, Elapsed Time, Arrival Delay = 1.80563%  
Cancellation Reason = 98.4554%  
Air System Delay, Security Delay, Airline Delay, Late Aircraft Delay, Weather Delay = 81.725%

##### Class balance  
This dataset will be splitted in two classes:  
**DELAYED**  
**NOT DELAYED**

Every flight having a departure delay greater than 0 will be considered as **delayed**, while having 0 or less departure delay is considered **not delayed**.

In [ ]:
df_delayed = df[df.DEPARTURE_DELAY > 0]
df_not_delayed = df[df.DEPARTURE_DELAY <= 0]

print "During 2015 there were {} fligths delayed and {} flights that were ahead of its departure time.".format(df_delayed.shape[0], df_not_delayed.shape[0])

In [ ]:
flights_delayed = df_delayed.shape[0]
flights_not_delayed = df_not_delayed.shape[0]
flights_total = df.shape[0]

print "In total there are {} flights delayed and {} flights on time on the dataset".format(flights_delayed,flights_not_delayed)
print "Distribution of classes is Delayed: {}% | Not Delayed {}%".format((flights_delayed * 100) / flights_total, (flights_not_delayed * 100) / flights_total)

In [ ]:
df['LABELS'] = np.where(df.DEPARTURE_DELAY > 0, 'DELAYED', 'NOT DELAYED')

In [ ]:
df.head()

### II.2 Cleaning the data

##### Cleaning unobtainable data
Since this project will be using live data to see if a flight will be delayed or not, it is not possible to use past data from flights, so every feature that relates to past information about the flight will be descarted.

DEPARTURE_TIME  
DEPARTURE_DELAY  
TAXI_OUT  
WHEELS_OFF  
ELAPSED_TIME  
AIT_TIME  
WHEELS_ON  
TAXI_IN  
ARRIVAL_TIME  
ARRIVAL_DELAY  
DIVERTED  
CANCELLED  
CANCELLATION_REASON  
AIR_SYSTEM_DELAY  
SECURITY_DELAY  
AIRLINE_DELAY  
LATE_AIRCRAFT_DELAY  
WEATHER_DELAY

In [ ]:
columns_to_drop = [
    'DEPARTURE_TIME',
    'DEPARTURE_DELAY',
    'TAXI_OUT',
    'WHEELS_OFF',
    'ELAPSED_TIME',
    'AIR_TIME',
    'WHEELS_ON',
    'TAXI_IN',
    'ARRIVAL_TIME',
    'ARRIVAL_DELAY',
    'DIVERTED',
    'CANCELLED',
    'CANCELLATION_REASON',
    'AIR_SYSTEM_DELAY',
    'SECURITY_DELAY',
    'AIRLINE_DELAY',
    'LATE_AIRCRAFT_DELAY',
    'WEATHER_DELAY'
]
df_clean = df.drop(columns=columns_to_drop)

##### Dealing with generalization vs specificity
Supposing the TAIL_NUMBER attribute is unique for each flight, let's verify how many unique tail numbers are present on the dataset and if it is unique for each entry then it can be discarted.

In [ ]:
tail_numbers = df.TAIL_NUMBER.unique()
print "In total there are {} unique tail numbers in {} flights on the dataset. ({}% of the records are unique)".format(tail_numbers.shape[0], df_clean.shape[0], ((tail_numbers.shape[0] * 100.00) / df_clean.shape[0]))

After counting the unique values it is clear that only a small part of the dataset is unique, which may give this feature a strong weight for the decission. Later in the project a feature selection will be executed to check if the feature helps as much in the decission making for the classifiers.

In [ ]:
flight_numbers = df.FLIGHT_NUMBER.unique()
print "In total there are {} unique flight numbers in {} flights on the dataset. ({}% of the records are unique)".format(flight_numbers.shape[0], df_clean.shape[0], ((flight_numbers.shape[0] * 100.00) / df_clean.shape[0]))

In [ ]:
print "Of the flights with tail number {}, there are only {} of {} unique flight numbers.".format('N407AS', df[df.TAIL_NUMBER == 'N407AS'].FLIGHT_NUMBER.unique().shape[0], df[df.TAIL_NUMBER == 'N407AS'].shape[0])

In [ ]:
df[(df.TAIL_NUMBER == 'N407AS') & (df.FLIGHT_NUMBER == 98)]

As shown above, the same plane did the same flight almost 4 times a year, but even when the same flight was performed it had different characteristics like date of week and hour of departure.

In [ ]:
null_info(df_clean)

##### Cleaning null datapoints
Now that the dataset has only the features that are available on classification time, it is possible to drop the rows containing null values since it will only loose less than 0.5% of the information, which in this case is not significant.

In [ ]:
df_clean = df_clean.drop(index=(df_clean[df_clean.TAIL_NUMBER.isnull()].index | df_clean[df_clean.SCHEDULED_TIME.isnull()].index))

In [ ]:
print "The final dataset contains {} rows and {} columns.".format(df_clean.shape[0], df_clean.shape[1])

In [ ]:
null_info(df_clean)

##### Understanding data domain

It is important to understand how the data is distributed between their domains, this is seeing which are the limits of the continuos features and the total number of classes in the discrete features.

In [ ]:
df_clean.dtypes

In [ ]:
def obj_to_cat(dataframe, column):
    dataframe[column] = dataframe[column].astype('category')

columns_to_categorize = [
    'AIRLINE',
    'FLIGHT_NUMBER',
    'TAIL_NUMBER',
    'ORIGIN_AIRPORT',
    'DESTINATION_AIRPORT',
    'LABELS'
]

for column in columns_to_categorize:
    obj_to_cat(df_clean, column)

In [ ]:
df_clean.dtypes

The feature YEAR contains the same value for all dataset, thus it can be removed since it won't make any difference.  
The MONTH and DAY features are numeric but they depend one another, it is not the same to be in the day 15th of February than the 15th of April, so it is possibly not optimal to take it as numerical.  
For this project the date will be considered as a combination of two classes, the week number and the day of the week. The combination of this two classes gives the same information as the date itself, and it is more generic than having the day number and the month.

In [ ]:
df_clean['DATE'] = df_clean.apply(lambda row: date(row.YEAR, row.MONTH, row.DAY), axis=1)

In [ ]:
df_clean['WEEK'] = df_clean.apply(lambda row: row.DATE.isocalendar()[1], axis=1)

In [ ]:
columns_to_categorize = [
    'WEEK',
    'DAY_OF_WEEK'
]

for column in columns_to_categorize:
    obj_to_cat(df_clean, column)

In [ ]:
columns_to_drop = [
    'YEAR',
    'MONTH',
    'DAY',
    'DATE'
]
df_clean = df_clean.drop(columns=columns_to_drop)

The only numeric features are:  
SCHEDULED_DEPARTURE  
SCHEDULED_TIME  
DISTANCE  
SCHEDULED_ARRIVAL  

Since each of them is measuring units (time in minutes and distance in kilometers). 
Every other feature is categorical since there is not a numerical scale between them, there are only classes to belong or not.

In [ ]:
df_clean.describe(include=[np.number])

In [ ]:
df_clean.describe(include=['category'])

The final dataset contains 11 features and 1 label, four of this features are continuos and 7 are discrete.

In [ ]:
df_clean.to_csv('./flights_clean.csv', index=False)